In [1]:
#Import all the dependencies
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize


In [2]:
import gensim
from gensim import corpora
from pprint import pprint
from gensim.utils import simple_preprocess

import pandas as pd

df = pd.read_csv("for_gensim.csv")

df.head()

,Date,Time,formtype,Improve,Imp1,Imp2,ImpCrit,CommentCoderImp,TweetImp,Best,Best1,Best2,BestCrit,Division2,Directorate2,Location
0,2011-10-01,11,adult,get treatment i requested,50.0,NaN,1.0,NaN,NaN,It helped.,40.0,NaN,1.0,Forensic services,Offender health,Offender health
1,2011-10-01,11,adult,provide gym staff to get gym up and running,47.0,NaN,1.0,NaN,NaN,Early stages waiting to see course contents an...,46.0,NaN,1.0,Forensic services,Offender health,Offender health
2,2011-10-01,11,adult,to look at the problems properly and give you ...,44.0,NaN,2.0,NaN,NaN,The best thing about it is getting treatment e...,61.0,NaN,1.0,Forensic services,Offender health,Offender health
3,2011-10-01,11,adult,to be more involved,27.0,NaN,2.0,NaN,NaN,Keep in on meds.,44.0,NaN,1.0,Forensic services,Offender health,Offender health
4,2011-10-01,11,adult,my drug problem was with subutex i was given m...,44.0,NaN,2.0,NaN,NaN,When I asked to see the doctor regarding my me...,61.0,NaN,1.0,Forensic services,Offender health,Offender health


In [3]:
# make a training and test set

# filter by date

df['Date'] = pd.to_datetime(df['Date'])

mask_training = (df['Date'] < "2018-01-01")
mask_test = (df['Date'] >= "2018-01-01")

df_training = df.loc[mask_training]
len(df_training)

df_test = df.loc[mask_test]
len(df_test)

comments_training = df_training["Improve"]
comments_test = df_test["Improve"]


description_training = comments_training.apply(lambda x: x.split()).tolist()
description_test = comments_test.apply(lambda x: x.split()).tolist()
print(description_training[1])
print(description_test[1])


['provide', 'gym', 'staff', 'to', 'get', 'gym', 'up', 'and', 'running']
['cant', 'give', 'any', 'feedback', 'on', 'what', 'could', 'be', 'done', 'better']


In [4]:
documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(description_training)]
model = Doc2Vec(documents, vector_size=5, window=2, min_count=1, workers=4)

In [5]:
from gensim.test.utils import get_tmpfile

fname = get_tmpfile("my_doc2vec_model")

model.save(fname)
model = Doc2Vec.load(fname)  # you can continue training with the loaded model!

In [26]:
# tokens = "a new sentence to match".split()

tokens = description_test[3]

new_vector = model.infer_vector(tokens)
sims = model.docvecs.most_similar([new_vector]) #gives you top 10 document tags and their cosine similarity
print(sims)

[(28961, 0.9924703240394592), (32561, 0.9910312294960022), (25700, 0.9893164038658142), (452, 0.9878360033035278), (39475, 0.9847462177276611), (26792, 0.9827353954315186), (3170, 0.9814635515213013), (4840, 0.9811143279075623), (2281, 0.9796568155288696), (8940, 0.977570116519928)]


In [27]:
res_list = [x[0] for x in sims]

print(description_test[3])

print("Similar comments:")

T = [comments_training[i] for i in res_list]

print(T)

#print(description_training[res_list])

['dining', 'room', 'floor', 'could', 'do', 'with', 'sweeping', 'more']
Similar comments:
['to know when groups come to an end so we can sign up for the next one', 'help with legs', 'more information available about babytoddler groups in the area and better website access to find info for new mums', 'all ok', 'everything was good', 'couldnt be better', 'more independence with medication stage 2', 'to control my anger more', 'to make me better', 'no all very good']
